Inertial frame of reference

In [4]:
import numpy as np
import numpy.linalg as la
from scipy.sparse.linalg import eigsh

# Experimental setup
M = 10. # g
C = 4. # N/m
w0 = C / M
a = 1. # distance between spheres
x = np.array([[1.], [0.]]) # x hat
y = np.array([[0.], [1.]]) # y hat
a1 = np.sqrt(3) * a * x
a2 = (np.sqrt(3) * x + 3 * y) * a / 2.
R1 = 1 / 3 * (a1 + a2)
R2 = 1 / 3 * (-2 * a1 + a2)
R3 = 1 / 3 * (a1 - 2 * a2)
R1h = R1 / la.norm(R1)
R2h = R2 / la.norm(R2)
R3h = R3 / la.norm(R3)
R11 = R1h * R1h.conj().T
R22 = R2h * R2h.conj().T
R33 = R3h * R3h.conj().T
kxs = np.arange(0, np.pi / np.sqrt(3) * 2, 0.1)


# System specific parameters
N = 9 # number of unit cells

evals_all, evecs_all  = [], []
for kx in kxs:
    k = np.array([kx, 0.])
    p = np.exp(1.j * k.dot(a1))

    # BUild Hamiltonian
    H = np.zeros((N * 2 * 2, N * 2 * 2), dtype=np.complex128)
    for i in range(0, N * 2 * 2 - 2, 2):
        if (i / 2) % 2 == 0:
            H[i:i+2, i+2:i+4] = -R11 - R22 * p
        else:
            H[i:i+2, i+2:i+4] = -R33
    H = H + H.conj().T
    for i in range(0, N * 2 * 2, 2):
        H[i:i+2, i:i+2] = R11 + R22 + R33

    # Solve eigenvalue equation
    evals, evecs = la.eigh(H)
    idcs = np.argsort(evals)
    evals, evecs = evals[idcs], evecs[idcs]
    evals_all.append(evals)
    evecs_all.append(evecs)
evals_all, evecs_all = np.array(evals_all, dtype=np.complex128), np.array(evecs_all, dtype=np.complex128)
print(evals_all.shape, evecs_all.shape)



# Plot
import plotly.graph_objects as go


data = []
for kx in kxs:
    num_eigs = evals_all.shape[1]
    for i in range(num_eigs):
        color = '#%02x%02x%02x' % (255 - int(i * 255 / num_eigs), 0, int(i * 255 / num_eigs))
        data.append(go.Scatter(x=kxs, y=(np.sqrt(evals_all[:, i]) * w0).real, name=f"{i}", line=dict(color=color)))

fig = go.Figure(data=data)
fig.show()


(37, 36) (37, 36, 36)


With coriolis force

In [28]:
import numpy as np
import numpy.linalg as la
from scipy.sparse.linalg import eigsh

# Experimental setup
M = 1.
C = 1.
w0 = C / M
a = 1. # distance between spheres
x = np.array([[1.], [0.]]) # x hat
y = np.array([[0.], [1.]]) # y hat
a1 = np.sqrt(3) * a * x
a2 = (np.sqrt(3) * x + 3 * y) * a / 2.
R1 = 1 / 3 * (a1 + a2)
R2 = 1 / 3 * (-2 * a1 + a2)
R3 = 1 / 3 * (a1 - 2 * a2)
R1h = R1 / la.norm(R1)
R2h = R2 / la.norm(R2)
R3h = R3 / la.norm(R3)
R11 = R1h * R1h.conj().T
R22 = R2h * R2h.conj().T
R33 = R3h * R3h.conj().T
kxs = np.arange(0, 3.55, 0.1)


# System specific parameters
N = 2 # number of unit cells

evals_all, evecs_all  = [], []
for kx in kxs:
    k = np.array([kx, 0.])
    p = np.exp(1.j * k.dot(a1))

    # BUild Hamiltonian
    H = np.zeros((N * 2 * 2, N * 2 * 2), dtype=np.complex128)
    for i in range(0, N * 2 * 2 - 2, 2):
        if (i / 2) % 2 == 0:
            H[i:i+2, i+2:i+4] = -R11 - R22 * p
        else:
            H[i:i+2, i+2:i+4] = -R33
    H = H + H.conj().T
    for i in range(0, N * 2 * 2, 2):
        H[i:i+2, i:i+2] = R11 + R22 + R33

    # Solve eigenvalue equation
    evals, evecs = la.eigh(H)
    idcs = np.argsort(evals)
    evals, evecs = evals[idcs], evecs[idcs]
    evals_all.append(evals)
    evecs_all.append(evecs)
evals_all, evecs_all = np.array(evals_all, dtype=np.complex128), np.array(evecs_all, dtype=np.complex128)
print(evals_all.shape, evecs_all.shape)

import sympy
a, b, w = sympy.symbols(['a', 'b', 'w'])

from scipy.optimize import fsolve


ev = evecs[1][6:8]
O = 1.
def equations(p):
    w, a, b = p
    return (w ** 2 * a - ev[0].real, 
            -2 * O * b * w - ev[0].imag,
            w ** 2 * b - ev[1].real)
    # 2 * O * a * w - ev[1].imag)

w, a, b = fsolve(equations, (1, 1, 1))
print(w, a, b)
print(w ** 2 * a - ev[0].real, 
            -2 * O * b * w - ev[0].imag,
            w ** 2 * b - ev[1].real)
print(2 * O * a * w - ev[1].imag)


# # Plot
# import plotly.graph_objects as go


# data = []
# for kx in kxs:
#     num_eigs = evals_all.shape[1]
#     for i in range(num_eigs):
#         color = '#%02x%02x%02x' % (255 - int(i * 255 / num_eigs), 0, int(i * 255 / num_eigs))
#         O = 20. # omega
#         ys = np.sqrt(2 * O ** 2 + np.sqrt(4 * O ** 2 + evals_all[:, i]))
#         data.append(go.Scatter(x=kxs, y=ys.real, name=f"{i}", line=dict(color=color)))
# fig = go.Figure(data=data)
# fig.show()

(36, 8) (36, 8, 8)
0.003621824699168719 -1.9511473493754734 -3.7809597058394027
-2.559439808179066e-05 -1.2739803616212964e-06 -4.9597170543547166e-05
-0.014133427323365673
